In [68]:
import datetime as dt

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestRegressor
import os
import glob

In [2]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [69]:
forest_2012_viirs = pd.read_csv('viirs-snpp_2012_United_States.csv')
forest_2013_viirs = pd.read_csv('viirs-snpp_2013_United_States.csv')
forest_2014_viirs = pd.read_csv('viirs-snpp_2014_United_States.csv')
forest_2015_viirs = pd.read_csv('viirs-snpp_2015_United_States.csv')
forest_2016_viirs = pd.read_csv('viirs-snpp_2016_United_States.csv')
forest_2017_viirs = pd.read_csv('viirs-snpp_2017_United_States.csv')
forest_2018_viirs = pd.read_csv('viirs-snpp_2018_United_States.csv')
forest_2019_viirs = pd.read_csv('viirs-snpp_2019_United_States.csv')
forest_2020_viirs = pd.read_csv('viirs-snpp_2020_United_States.csv')
forest_2021_viirs = pd.read_csv('viirs-snpp_2021_United_States.csv')


C:\Users\prera\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [70]:
print("start viirs")
# Here we can see that we have 36011 rows and 15 columns in our dataset, obviously we have to do lot of data cleaning but first let's explore this dataset more
print(forest_2012_viirs.shape)
print(forest_2013_viirs.shape)
print(forest_2014_viirs.shape)
print(forest_2015_viirs.shape)
print(forest_2016_viirs.shape)
print(forest_2017_viirs.shape)
print(forest_2018_viirs.shape)
print(forest_2019_viirs.shape)
print(forest_2020_viirs.shape)
print(forest_2021_viirs.shape)

start viirs
(531867, 15)
(401694, 15)
(373222, 15)
(577122, 15)
(428949, 15)
(593047, 15)
(552123, 15)
(447311, 15)
(672450, 15)
(661058, 15)


In [72]:
forest = pd.concat([forest_2012_viirs,
           forest_2013_viirs,
           forest_2014_viirs,
           forest_2015_viirs,
           forest_2016_viirs,
           forest_2017_viirs,
           forest_2018_viirs,
           forest_2019_viirs,
           forest_2020_viirs,
           forest_2021_viirs], ignore_index=True)

In [75]:
print(forest.shape)
forest.head()

(5238843, 15)


,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,type
0,40.393394,-79.854332,334.52,0.46,0.63,2012-01-20,619,N,VIIRS,n,1,265.82,5.02,N,2
1,39.214512,-76.478729,328.69,0.59,0.53,2012-01-20,619,N,VIIRS,n,1,271.66,5.20,N,2
2,39.217484,-76.477531,328.30,0.59,0.53,2012-01-20,619,N,VIIRS,n,1,272.87,5.50,N,2
3,35.479481,-76.787148,296.78,0.57,0.52,2012-01-20,620,N,VIIRS,n,1,273.75,1.15,N,0
4,36.356388,-76.808456,298.89,0.58,0.52,2012-01-20,620,N,VIIRS,n,1,275.99,1.21,N,3


In [76]:
forest.columns

Index(['latitude', 'longitude', 'bright_ti4', 'scan', 'track', 'acq_date',
       'acq_time', 'satellite', 'instrument', 'confidence', 'version',
       'bright_ti5', 'frp', 'daynight', 'type'],
      dtype='object')

In [81]:
# check null values
forest.isnull().sum()

latitude      0
longitude     0
bright_ti4    0
scan          0
track         0
acq_date      0
acq_time      0
satellite     0
instrument    0
confidence    0
version       0
bright_ti5    0
frp           0
daynight      0
type          0
dtype: int64

In [82]:
forest.describe()

,latitude,longitude,bright_ti4,scan,track,acq_time,version,bright_ti5,frp,type
count,5.238843e+06,5.238843e+06,5.238843e+06,5.238843e+06,5.238843e+06,5.238843e+06,5238843.0,5.238843e+06,5.238843e+06,5.238843e+06
mean,3.941158e+01,-1.081078e+02,3.279809e+02,4.626892e-01,4.858153e-01,1.424782e+03,1.0,2.923263e+02,1.328326e+01,4.081092e-01
std,8.946629e+00,1.997561e+01,2.128128e+01,9.017033e-02,1.181291e-01,5.590897e+02,0.0,1.260395e+01,4.189050e+01,8.833394e-01
min,1.891902e+01,-1.788074e+02,2.080000e+02,3.200000e-01,3.600000e-01,0.000000e+00,1.0,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.343828e+01,-1.205183e+02,3.073300e+02,3.900000e-01,3.800000e-01,9.120000e+02,1.0,2.850400e+02,1.690000e+00,0.000000e+00
50%,3.881353e+01,-1.085260e+02,3.307600e+02,4.400000e-01,4.500000e-01,1.233000e+03,1.0,2.911500e+02,4.010000e+00,0.000000e+00
75%,4.377722e+01,-9.118479e+01,3.432400e+02,5.100000e-01,5.700000e-01,1.942000e+03,1.0,2.984700e+02,9.830000e+00,0.000000e+00
max,7.132837e+01,1.795882e+02,3.670000e+02,8.000000e-01,7.800000e-01,2.359000e+03,1.0,3.800000e+02,8.065420e+03,3.000000e+00


In [83]:
# Finding categorical data
print("The satellite column")
print(forest['satellite'].value_counts())
print()
print("The aqc_time column")
print(forest['acq_time'].value_counts())
print()
print("The satellite column")
print(forest['satellite'].value_counts())
print()
print("The instrument column")
print(forest['instrument'].value_counts())
print()
print("The version column")
print(forest['version'].value_counts())
print()
print("The daynight column")
print(forest['daynight'].value_counts())
print()
print("The type column")
print(forest['type'].value_counts())
print()
print("The confidence column")
print(forest['confidence'].value_counts())
print()

The satellite column
N    5238843
Name: satellite, dtype: int64

The aqc_time column
942     39296
1024    37357
1000    36694
900     36313
1006    35145
        ...  
1643        1
544         1
1505        1
1623        1
1548        1
Name: acq_time, Length: 1151, dtype: int64

The satellite column
N    5238843
Name: satellite, dtype: int64

The instrument column
VIIRS    5238843
Name: instrument, dtype: int64

The version column
1    5238843
Name: version, dtype: int64

The daynight column
D    2389232
N    2297112
0     320025
1     221353
1       5951
0       5170
Name: daynight, dtype: int64

The type column
0    4270932
2     687993
3     241058
1      38860
Name: type, dtype: int64

The confidence column
n    4705233
h     332743
l     200867
Name: confidence, dtype: int64



In [84]:
forest = forest.drop(['satellite', 'instrument', 'version'], axis = 1)

In [85]:
forest.head()

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,confidence,bright_ti5,frp,daynight,type
0,40.393394,-79.854332,334.52,0.46,0.63,2012-01-20,619,n,265.82,5.02,N,2
1,39.214512,-76.478729,328.69,0.59,0.53,2012-01-20,619,n,271.66,5.20,N,2
2,39.217484,-76.477531,328.30,0.59,0.53,2012-01-20,619,n,272.87,5.50,N,2
3,35.479481,-76.787148,296.78,0.57,0.52,2012-01-20,620,n,273.75,1.15,N,0
4,36.356388,-76.808456,298.89,0.58,0.52,2012-01-20,620,n,275.99,1.21,N,3


In [86]:
daynight_map = {"D": 1, "N": 0}
confidence_map = {"n": 0, "l": 1, "h": 2}

forest['daynight'] = forest['daynight'].map(daynight_map)
forest['confidence'] = forest['confidence'].map(confidence_map)

forest.head()

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,confidence,bright_ti5,frp,daynight,type
0,40.393394,-79.854332,334.52,0.46,0.63,2012-01-20,619,0,265.82,5.02,0.0,2
1,39.214512,-76.478729,328.69,0.59,0.53,2012-01-20,619,0,271.66,5.20,0.0,2
2,39.217484,-76.477531,328.30,0.59,0.53,2012-01-20,619,0,272.87,5.50,0.0,2
3,35.479481,-76.787148,296.78,0.57,0.52,2012-01-20,620,0,273.75,1.15,0.0,0
4,36.356388,-76.808456,298.89,0.58,0.52,2012-01-20,620,0,275.99,1.21,0.0,3


In [87]:
forest['acq_date'] = pd.to_datetime(forest['acq_date'])

forest['month'] = forest['acq_date'].dt.month
forest['day'] = forest['acq_date'].dt.day

y = forest['confidence']
fin = forest.drop(['confidence', 'acq_date', 'acq_time'], axis = 1)

In [88]:
fin.head()

print("The day column")
print(forest['day'].value_counts())
print()
print("The month column")
print(forest['month'].value_counts())
print()
print("The type column")
print(forest['type'].value_counts())
print()
print("The daynight column")
print(forest['daynight'].value_counts())
print()


The day column
9     203576
8     201341
10    185994
20    183780
7     183144
19    179663
23    178099
22    178080
6     177841
5     174124
11    174050
15    173203
18    172194
25    171245
21    170122
24    170028
4     169095
17    168976
16    167858
14    167445
3     163763
13    162955
12    162795
2     161990
29    160961
28    160184
1     157867
26    155425
30    152444
27    147772
31    102829
Name: day, dtype: int64

The month column
8     1042627
9      819838
7      675745
6      463451
3      398987
10     398140
4      380159
5      277117
11     266531
2      196990
12     159763
1      159495
Name: month, dtype: int64

The type column
0    4270932
2     687993
3     241058
1      38860
Name: type, dtype: int64

The daynight column
1.0    2389232
0.0    2297112
Name: daynight, dtype: int64



In [91]:
%%time

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(fin, y, test_size = 0.2, random_state = 0)

from xgboost import XGBClassifier

classifier = XGBClassifier()
classifier.fit(X_train, y_train)

[20:38:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Wall time: 10min 59s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [92]:
%%time

from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[934471   6311      0]
 [  6291  30824   3242]
 [     0   1576  65054]]
Wall time: 701 ms


0.9833741979386678

In [93]:
%%time

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

C:\Users\prera\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:49:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\prera\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:57:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\prera\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[21:50:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\prera\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:59:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\prera\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:07:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 98.35 %
Standard Deviation: 0.02 %
Wall time: 1h 27min 59s
